### Contents

* Guidelines for Prompting
  - Write clear and specific instructions 
  - Give the model time to "think"
  
* Iterative prompt development
* Capabilities 
  - **Summarizing**
  - Infering
  - Transforming
  - Expanding
* Building a Chatbot

# Summarizing

이 단원에서는 특정 주제에 중점을 두고 텍스트를 요약합니다.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 이것는 모델 출력의 무작위성 정도를 말합니다.[0~1]
    )
    return response.choices[0].message.content

## Text to summarize

In [6]:
prod_review = """
내 딸의 생일을 위해 이 팬더 플러시 장난감을 샀는데, 
딸은 그것을 좋아하고 어디든 갖고 다닙니다. 
부드럽고 매우 귀엽고, 얼굴이 친근해 보입니다. 
그래도 내가 낸 금액에 비해 좀 작습니다. 같은 가격에 
더 큰 옵션이 있을 수도 있다고 생각합니다. 
예정보다 하루 일찍 도착해서 선물하기 전에 제가 직접 가지고 놀게 되었어요.
"""

## Summarize with a word/sentence/character limit

In [7]:
prompt = f"""
당신의 임무는 전자상거래 사이트에서 제품 리뷰에 대한 \
간략한 요약을 생성하는 것입니다.

아래 삼중 백틱으로 구분된 리뷰를 최대 30단어로 요약하세요.

리뷰:```{prod_review}```
"""

response = get_completion(prompt)
print(response)

팬더 플러시 장난감은 딸이 좋아하며 부드럽고 귀엽지만 가격에 비해 작을 수 있고 더 큰 옵션이 있을 수도 있습니다.


In [8]:
len(response.split(" "))

18

## Summarize with a focus on shipping and delivery

In [9]:
prompt = f"""
당신의 임무는 전자상거래 사이트에서 제품 리뷰에 대한 
간략한 요약을 생성하여 배송 부서에 피드백을 제공하는 것입니다.

세 개의 역따옴표로 구분된 리뷰를 
제품 배송 및 배달과 관련된 모든 측면에 중점을 두어
최대 30단어로 요약합니다. 

리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

요약: "팬더 플러시 장난감은 딸이 좋아하며 부드럽고 귀엽지만 가격 대비 작을 수 있고 더 큰 옵션이 있을 수도 있습니다. 예상보다 일찍 도착하여 직접 확인할 수 있었습니다."


## Summarize with a focus on price and value

In [10]:
prompt = f"""
당신의 귀하의 임무는 전자상거래 사이트에서 제품 리뷰에 대한 
간략한 요약을 생성하여 제품 가격 결정을 담당하는 가격 책정 부서에 피드백을 제공하는 것입니다.

세 개의 역따옴표로 구분된 리뷰를 
가격 및 인지된 가치와 관련된 모든 측면에 초점을 맞추어
최대 30단어로 요약합니다, 
""
리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

"팬더 플러시 장난감은 딸이 좋아하고 매우 귀엽지만 가격 대비 크기가 작을 수 있음. 더 큰 옵션을 고려할 수 있으며 빠른 배송."


## Try "extract" instead of "summarize"

In [11]:
prompt = f"""
당신의 임무는 전자상거래 사이트의 제품 리뷰에서 관련 정보를 추출하여 \
배송 부서에 피드백을 제공하는 것입니다.

아래 세 개의 역따옴표로 구분된 리뷰에서 배송 및 배송과 관련된 정보를 추출합니다. 
최대 30단어로 요약합니다, 

리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


요약: "팬더 플러시 장난감은 딸이 좋아하며 부드럽고 귀엽지만 가격에 비해 작을 수 있음. 예정보다 하루 일찍 도착하여 직접 확인함."


## Summarize multiple product reviews

In [12]:
review_1 = prod_review

# review for a standing lamp
review_2 = """
내 침실에 사용할 멋진 램프가 필요했는데, 
이 램프는 추가 저장 공간을 제공하고 가격도 그리 높지 않습니다. 
빠르게 받았습니다 - 2일만에 도착했습니다. 
운송 중에 램프의 끈이 끊어졌고 회사에서는 기꺼이 새 끈을 보내주었습니다. 
그것도 며칠 안에 오더군요. 조립하기 쉬웠어요. 
그러다가 누락된 부품이 있어서 지원팀에 연락했더니 \
누락된 부품을 매우 빨리 받았습니다! 
제가 보기엔 고객과 제품에 관심을 갖는 훌륭한 회사인 것 같습니다.
"""

# review for an electric toothbrush
review_3 = """
치과위생사가 전동칫솔을 추천해서 이걸 샀어요. 
배터리 수명은 지금까지 꽤 인상적인 것 같습니다. 
초기 충전 후 배터리 상태를 조절하기 위해 첫 주 동안 충전기를 \
연결한 상태로 두었습니다. 
저는 충전기의 플러그를 뽑고 지난 3주 동안 매일 2번씩 \
동일한 충전으로 양치를 사용했습니다. 
그런데 칫솔머리가 너무 작아요. 이것보다 더 큰 유아용 칫솔을 본 적이 있어요. 
머리가 더 크고 길이가 다른 강모를 사용하여 치아 사이에 더 잘 \
들어갈 수 있기를 바랍니다. 
왜냐하면 이것은 그렇지 않기 때문입니다. 
전반적으로 이 제품을 50달러 정도에 구입할 수 있다면 좋은 거래입니다. 
제조업체의 교체용 헤드는 꽤 비싸지만 보다 합리적인 가격으로 \
일반 헤드를 구입할 수 있습니다. 
이 칫솔을 사용하면 매일 치과에 다녀온 것 같은 느낌이 듭니다. 
치아가 반짝반짝 깨끗한 느낌이에요!
"""

# review for a blender
review_4 = """
그래서 그들은 여전히 11월 한 달 동안 약 $49에 시즌 세일로 17개 시스템을 
절반 정도 할인된 가격으로 판매하고 있었지만 어떤 이유에서든(가격 바가지라고 부르죠) 
12월 두 번째 주쯤에 가격이 모두 20~30%까지 올랐습니다.
동일한 시스템의 경우 $70-$89 사이입니다. 
그리고 11피스 시스템의 가격도 이전 판매 가격인 29달러에서 약 10달러 정도 올랐습니다. 
그래서 괜찮아 보이는데, 베이스를 보면 칼날이 고정되는 부분이 
몇년 전의 전작들만큼 좋지는 않은데, 아주 부드럽게 처리할 예정입니다. 
콩, 얼음, 쌀 등과 같이 매우 딱딱한 것을 믹서기에 먼저 분쇄한 다음 
믹서기에 넣고 원하는 크기로 분쇄한 다음 밀가루를 더 곱게 만들려면 
휘핑 칼날로 전환하고, 스무디를 만들 때는 십자 절단 칼날을 먼저 사용하십시오. , 
더 미세하고 덜 과육이 필요한 경우 플랫 블레이드를 사용하십시오. 
스무디를 만들 때 특별한 팁, 사용할 과일과 야채를 잘게 잘라 냉동하세요
(시금치를 살짝 끓여서 시금치를 부드럽게 만든 후 사용할 준비가 될 때까지 얼리고, 
셔벗을 만들 경우 중소형 푸드 프로세서를 사용하세요). 
그렇게 하면 스무디를 만들 때 너무 많은 얼음을 추가하는 것을 피할 수 있습니다. 
1년 정도 지나자 모터에서 이상한 소리가 나기 시작했습니다. 
고객센터에 전화했는데 이미 보증기간이 만료되어 하나 더 구입해야 했어요. 
참고: 이러한 유형의 제품에서는 전반적인 품질이 향상되었으므로 
판매를 유지하기 위해 브랜드 인지도와 소비자 충성도에 의존하고 있습니다. 
이틀 정도만에 받았습니다.
"""

reviews = [review_1, review_2, review_3, review_4]

In [13]:
for i in range(len(reviews)):
    prompt = f"""
    귀하의 임무는 전자상거래 사이트에서 제품 리뷰에 대한\
    간략한 요약을 생성하는 것입니다.

    삼중 백틱으로 구분된 아래 리뷰를 최대 20단어로 요약하세요. 

    리뷰: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 팬더 플러시 장난감은 딸이 좋아하고 매우 귀엽지만 가격에 비해 작을 수 있음. 

1 요약: 멋진 램프, 추가 저장 공간, 빠른 배송, 친절한 고객 서비스, 쉬운 조립, 누락된 부품 신속 대응, 훌륭한 회사. 

2 요약: 전동칫솔의 배터리 수명이 좋고 유아용 칫솔보다 작지만 효과적이며 가격 대비 좋은 제품. 

3 요약: 11월에 할인된 시스템 가격이 12월에 20-30% 올랐고, 다양한 칼날로 다양한 식재료를 처리할 수 있으며, 모터 소리 문제 발생 후 보증기간 만료로 새 제품 구매 필요. 



## Try experimenting on your own!